# IIR Filter -Chebyshev Filters

Chebyshev filters are used to separate one band of frequencies from another. Although they cannot match the performance of the windowed-sinc filter, they are more than adequate for many applications. The primary attribute of Chebyshev filters is their speed, typically more than an order of magnitude faster than the windowed-sinc. This is because they are carried out by recursion rather than convolution. The design of these filters is based on the z-transform.

## The Chebyshev and Butterworth Responses
The Chebyshev response is a mathematical strategy for achieving a faster roll-off by allowing ripple in the frequency response. Analog and digital filters that use this approach are called Chebyshev filters. These filters are named from their use of the Chebyshev polynomials, developed by the Russian mathematician Pafnuti Chebyshev (1821-1894).

The Figure shows the frequency response of low-pass Chebyshev filters with passband ripples of: 0%, 0.5% and 20%. As the ripple increases (bad), the roll-off becomes sharper (good). The Chebyshev response is an optimal trade-off between these two parameters. When the ripple is set to 0%, the filter is called a maximally flat or Butterworth filter (after S. Butterworth, a British engineer who described this response in 1930). A ripple of 0.5% is a often good choice for digital filters. This matches the typical precision and accuracy of the analog electronics that the signal has passed through.

![Chebyshev Roll Off](Images/Chebyshev_Roll_Off.gif)

In [ ]:
import sys
sys.path.insert(0, '../')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from Common import common_plots
from Common import fourier_inverse_transform
from Common import fourier_transform
cplots = common_plots.Plot()
from Common import digital_filter
from scipy.signal import cheby1

## Designing the Filter
You must select four parameters to design a Chebyshev filter: 
1. A high-pass or low-pass response.
2. The cutoff frequency.
3. The allowed ripple in the passband.
4. The number of poles. 

We will design a 6th order low pass Chebyshev filter with a cutoff frequency of 16Hz for a signal that has been sampled at 64Hz. The allowed ripple for this filter is 0.1 dB.

In [ ]:
# Sampling parameters
fs = None  # Hz

# Desired filter parameters
order = None
ripple = None  # dB
fcut = None    # Hz

# Normalized frequency argument for cheby1
wn = None

# btype arguments can be 'lowpass', 'highpass', 'bandpass' or 'bandstop'
a, b = None

In [ ]:
#We need to put our coefficients in our programming style
c = np.flip(a)
d = np.flip(b)

w = np.arange(0, np.pi, 0.01)

In [ ]:
H_w = digital_filter.filter_frequency_response(c, d, w)
idft = fourier_inverse_transform.FourierInverseTransform(H_w, np.zeros(len(H_w)))
z, p, g = digital_filter.zeros_poles_gain(c, d)

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

plt.subplot(1, 3, 1)
plt.plot(w/(2*max(w)), 20*np.log(np.absolute(H_w)))
plt.title('Frequency Response')
plt.xlabel('frequency')
plt.ylabel('dB')
plt.grid('on')

plt.subplot(1, 3, 2)
plt.stem(np.real(idft.synth[0:25]), use_line_collection=True)
plt.title('Impulse Response')
plt.xlabel('sample')
plt.grid('on')

cplots.plot_zeros_poles(z, p)
plt.title('Zeros and Poles')
plt.xlabel('sample');

In [ ]:
plt.plot(w[0:200]/(2*max(w)), 20*np.log(np.absolute(H_w[0:200])))
plt.title('Frequency Response')
plt.xlabel('frequency')
plt.ylabel('dB')
plt.grid('on')

## Using the Filter
We will use the designed filter by implementing the difference equation

$$ y[n] = a_0 x[n] + a_1 x[n-1] + a_2 x[n-2] + \cdot \cdot \cdot + b_1 y[n-1] + b_2 y[n-2] + \cdot \cdot \cdot $$ 

as a **Direct II Transposed Structure**.

The following Figure shows an example of a direct II transpose structure.

![Direct II Transposed Structure](Images/Transposed_Direct_Form_II.png)

As it can bee seen in the Figure, a state variable $s[n]$ is introduced. 

The following code shows how we can implement the direct II transpose structure.

``` python
     y[n] = a[0] * x[n]               + s[0][n-1]
  s[0][n] = a[1] * x[n] - b[1] * y[n] + s[1][n-1]
  s[1][n] = a[2] * x[n] - b[2] * y[n] + s[2][n-1]
...
s[N-2][n] = a[N-1]*x[n] - b[N-1]*y[n] + s[N-1][n-1]
s[N-1][n] = a[N] * x[n] - b[N] * y[n]

```

In [ ]:
def filter_dtype_ii(a, b, x):
    """ 
    Direct II Transposed Structure implementation of digital filter.
  
    Parameters: 
    a (numpy array): Array of numerator filter coefficients.
    b (numpy array): Array of denominator filter coefficients.
    x (numpy array): Array of signal of interest.
  
    Returns: 
    numpy array: Returns filter response.
  
    """
    
    return None

### Generating a signal for testing the filter
After you successfully created the Chebyshev filter, it is time to test it. For this purpose, we will create a dummy signal sampled at 64Hz that has two frequency components of 20 and 16Hz. Since our filter is a low pass one with a cutoff frequency of 16Hz, we should be able to get rid of the 20Hz component.

In [ ]:
fs = 64

fc1 = 20
fc2 = 16

t = np.arange(0,2,1/fs)

x1 = np.sin(2*np.pi*fc1*t)
x2 = np.sin(2*np.pi*fc2*t)
x = x1 + x2

dft = fourier_transform.FourierTransform(x.reshape(-1,1))
X = dft.dft_magnitude()
f = dft.frequency_domain(fsamp=64, style='analog')

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

plt.subplot(1,2,1)
plt.plot(t,x)
plt.title('Signal')
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.grid('on')

plt.subplot(1,2,2)
cplots.plot_frequency_response(X, f, title='Signal\'s FFT')


To test our filter function`filter_dtype_ii` we can compare it's results against the `lfilter` provided by SciPy.

In [ ]:
# Add your code here
y_filter = None

from scipy.signal import lfilter
y_scipy = lfilter(a, b, x)

# Fourier Transform of our filter implementation
dft_filter = fourier_transform.FourierTransform(y_filter.reshape(-1,1))
Y_filter = dft_filter.dft_magnitude()
F_filter = dft_filter.frequency_domain(fsamp=64, style='analog')

# Fourier Transform of SciPy implementation
dft_scipy = fourier_transform.FourierTransform(y_scipy.reshape(-1,1))
Y_scipy = dft_scipy.dft_magnitude()
F_scipy = dft_scipy.frequency_domain(fsamp=64, style='analog')

In [ ]:
plt.rcParams["figure.figsize"] = (15,5)

plt.subplot(1,2,1)
plt.plot(t,y_filter, label='implemented')
plt.plot(t,y_scipy, label='SciPy')
plt.title('Signal')
plt.xlabel('time')
plt.ylabel('Amplitude')
plt.grid('on')
plt.legend();

plt.subplot(1,2,2)
cplots.plot_frequency_response(Y_filter, F_filter, title='Signal\'s FFT', label='implemented')
cplots.plot_frequency_response(Y_scipy, F_scipy, title='Signal\'s FFT', label='SciPy')
plt.legend();

### Reference
[1] https://www.dspguide.com/ch20/2.htm <br>
[2] https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.cheby1.html <br>
[3] https://www.dsprelated.com/freebooks/filters/Four_Direct_Forms.html <br>
[4] https://www.dsprelated.com/freebooks/filters/Transposed_Direct_Forms.html